In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

In [ ]:
from awe import utils, qa
import awe.qa.trainer
utils.reload('awe.qa')

In [ ]:
# Reload QA modules.
import importlib
from awe import qa_model
from awe.data import qa_dataset
for module in [qa_model, qa_dataset]:
    importlib.reload(module)

In [ ]:
params = qa.trainer.QaTrainerParams(
    batch_size=1,
    train_subset=100,
    version_name='qa-bigger-batch',
    epochs=3,
)
params

In [ ]:
trainer = qa.trainer.QaTrainer(params)

In [ ]:
trainer.params = params

In [ ]:
trainer.load_pipeline()

In [ ]:
trainer.load_data()

In [ ]:
#trainer.prepare_data()

In [ ]:
trainer.create_model()

In [ ]:
trainer.restore('logs/63-qa-bigger-batch/checkpoints/epoch=2-step=299.ckpt')

In [ ]:
trainer.validate()

In [ ]:
trainer.validate_seen()

In [ ]:
[p.file_path for p in trainer.val_pages[:2]]

In [ ]:
trainer.predict_examples()

In [ ]:
trainer.predict_seen_examples()